# 0 - Dividing the dataset_rares_species into train,val,test (we run this once)

In [20]:
'''

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
drive_root = '/content/drive/MyDrive/Colab Notebooks/Deep Learning'
dataset_dir = os.path.join(drive_root, "dataset_rares_species")  # Folder with raw images
metadata_path = os.path.join(drive_root, "species_metadata.csv")  # Path to CSV
output_dir = os.path.join(drive_root, "data")  # Where to save splits (train/val/test)

# Create output directories
os.makedirs(os.path.join(output_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "val"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "test"), exist_ok=True)

'''

'\n\n# Mount Google Drive\ndrive.mount(\'/content/drive\')\n\n# Define paths\ndrive_root = \'/content/drive/MyDrive/Colab Notebooks/Deep Learning\'\ndataset_dir = os.path.join(drive_root, "dataset_rares_species")  # Folder with raw images\nmetadata_path = os.path.join(drive_root, "species_metadata.csv")  # Path to CSV\noutput_dir = os.path.join(drive_root, "data")  # Where to save splits (train/val/test)\n\n# Create output directories\nos.makedirs(os.path.join(output_dir, "train"), exist_ok=True)\nos.makedirs(os.path.join(output_dir, "val"), exist_ok=True)\nos.makedirs(os.path.join(output_dir, "test"), exist_ok=True)\n\n'

In [21]:
'''
# Load CSV
df = pd.read_csv(metadata_path)

# Split into train/val/test (stratified by family)
train_val_df, test_df = train_test_split(
    df,
    test_size=0.15,
    stratify=df['family'],
    random_state=42
)

train_df, val_df = train_test_split(
    train_val_df,
    test_size=0.15/0.85,  # Val = 15% of total
    stratify=train_val_df['family'],
    random_state=42
)
'''

"\n# Load CSV\ndf = pd.read_csv(metadata_path)\n\n# Split into train/val/test (stratified by family)\ntrain_val_df, test_df = train_test_split(\n    df,\n    test_size=0.15,\n    stratify=df['family'],\n    random_state=42\n)\n\ntrain_df, val_df = train_test_split(\n    train_val_df,\n    test_size=0.15/0.85,  # Val = 15% of total\n    stratify=train_val_df['family'],\n    random_state=42\n)\n"

In [22]:
'''
def copy_images(split_df, split_name):
    """Copy images to train/val/test folders using metadata."""
    for _, row in tqdm(split_df.iterrows(), desc=f"Copying {split_name} images"):
        # Source path: Parse from 'file_path' column
        src_path = os.path.join(dataset_dir, row['file_path'])

        # Destination path: Use 'family' as class folder
        dest_dir = os.path.join(output_dir, split_name, row['family'])
        os.makedirs(dest_dir, exist_ok=True)

        # Extract filename from 'file_path'
        filename = os.path.basename(row['file_path'])
        dest_path = os.path.join(dest_dir, filename)

        # Copy file
        if os.path.exists(src_path):
            copyfile(src_path, dest_path)
        else:
            print(f"Warning: {src_path} not found")

# Copy images
copy_images(train_df, "train")
copy_images(val_df, "val")
copy_images(test_df, "test")
'''

'\ndef copy_images(split_df, split_name):\n    """Copy images to train/val/test folders using metadata."""\n    for _, row in tqdm(split_df.iterrows(), desc=f"Copying {split_name} images"):\n        # Source path: Parse from \'file_path\' column\n        src_path = os.path.join(dataset_dir, row[\'file_path\'])\n\n        # Destination path: Use \'family\' as class folder\n        dest_dir = os.path.join(output_dir, split_name, row[\'family\'])\n        os.makedirs(dest_dir, exist_ok=True)\n\n        # Extract filename from \'file_path\'\n        filename = os.path.basename(row[\'file_path\'])\n        dest_path = os.path.join(dest_dir, filename)\n\n        # Copy file\n        if os.path.exists(src_path):\n            copyfile(src_path, dest_path)\n        else:\n            print(f"Warning: {src_path} not found")\n\n# Copy images\ncopy_images(train_df, "train")\ncopy_images(val_df, "val")\ncopy_images(test_df, "test")\n'

Copying train images: 464it [07:56,  1.19it/s]Warning: /content/drive/MyDrive/Colab Notebooks/Deep Learning/dataset_rares_species/arthropoda_apidae/28260824_1065329_eol-full-size-copy.jpg not found
Copying train images: 5104it [1:12:41,  1.17it/s]Warning: /content/drive/MyDrive/Colab Notebooks/Deep Learning/dataset_rares_species/arthropoda_apidae/28214440_1065290_eol-full-size-copy.jpg not found
Copying train images: 7311it [1:43:10,  1.19it/s]Warning: /content/drive/MyDrive/Colab Notebooks/Deep Learning/dataset_rares_species/arthropoda_apidae/28260831_1065329_eol-full-size-copy.jpg not found
Copying train images: 7968it [1:52:16,  1.18it/s]Warning: /content/drive/MyDrive/Colab Notebooks/Deep Learning/dataset_rares_species/arthropoda_apidae/28214384_1065290_eol-full-size-copy.jpg not found
Copying train images: 8387it [1:58:04,  1.18it/s]
Copying val images: 626it [08:41,  1.31it/s]Warning: /content/drive/MyDrive/Colab Notebooks/Deep Learning/dataset_rares_species/arthropoda_apidae/22375122_1065346_eol-full-size-copy.jpg not found
Copying val images: 1798it [24:38,  1.22it/s]
Copying test images: 98it [01:21,  1.15it/s]Warning: /content/drive/MyDrive/Colab Notebooks/Deep Learning/dataset_rares_species/arthropoda_apidae/28408134_1065346_eol-full-size-copy.jpg not found
Copying test images: 1798it [24:39,  1.22it/s]

#0 - Creating a subset of the data do the tests (we run this once)

In [23]:
'''
import os
import random
import shutil
from pathlib import Path
from tqdm import tqdm

# Define paths
drive_root = '/content/drive/MyDrive/Colab Notebooks/Deep Learning'
data_dir = os.path.join(drive_root, "data")  # Where your splits are stored
exp_dir = os.path.join(drive_root, "experimentation")  # Where to save smaller dataset

# Define target sizes
train_size = 1676
val_size = 358
test_size = 358

# Set random seed for reproducibility
random.seed(42)

# Create experimentation directories
os.makedirs(os.path.join(exp_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(exp_dir, "val"), exist_ok=True)
os.makedirs(os.path.join(exp_dir, "test"), exist_ok=True)

def create_subset(src_dir, dest_dir, target_size):
    """Create a stratified subset of images maintaining class proportions."""

    # Get all class folders
    class_folders = [f for f in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, f))]

    # Count total images
    total_images = 0
    class_counts = {}
    for cls in class_folders:
        cls_path = os.path.join(src_dir, cls)
        image_files = [f for f in os.listdir(cls_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        class_counts[cls] = len(image_files)
        total_images += len(image_files)

    # Calculate how many images to take from each class
    class_samples = {}
    remaining = target_size
    for cls, count in class_counts.items():
        # Calculate proportional number (with at least 1 image per class)
        samples = max(1, int(count * target_size / total_images))
        class_samples[cls] = min(samples, count)  # Can't sample more than available
        remaining -= class_samples[cls]

    # Adjust to match exact target size
    if remaining > 0:
        # Distribute remaining samples to classes with more images
        classes_sorted = sorted(class_counts.keys(), key=lambda c: class_counts[c], reverse=True)
        for cls in classes_sorted:
            if class_samples[cls] < class_counts[cls]:
                class_samples[cls] += 1
                remaining -= 1
            if remaining == 0:
                break
    elif remaining < 0:
        # Remove samples to match target size
        classes_sorted = sorted(class_counts.keys(), key=lambda c: class_counts[c], reverse=True)
        for cls in classes_sorted:
            if class_samples[cls] > 1:  # Keep at least 1 sample
                class_samples[cls] -= 1
                remaining += 1
            if remaining == 0:
                break

    # Copy images
    for cls in tqdm(class_folders, desc=f"Creating subset in {os.path.basename(dest_dir)}"):
        src_class_dir = os.path.join(src_dir, cls)
        dest_class_dir = os.path.join(dest_dir, cls)
        os.makedirs(dest_class_dir, exist_ok=True)

        # Get all image files in this class
        image_files = [f for f in os.listdir(src_class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Randomly sample the required number
        samples_to_take = min(class_samples[cls], len(image_files))
        sampled_files = random.sample(image_files, samples_to_take)

        # Copy sampled files
        for file in sampled_files:
            src_file = os.path.join(src_class_dir, file)
            dest_file = os.path.join(dest_class_dir, file)
            shutil.copy2(src_file, dest_file)

    # Count total copied
    copied_total = 0
    for cls in class_folders:
        dest_class_dir = os.path.join(dest_dir, cls)
        if os.path.exists(dest_class_dir):
            copied_total += len(os.listdir(dest_class_dir))

    print(f"Copied {copied_total} images to {os.path.basename(dest_dir)}")
    return copied_total

# Create subsets for each split
print("Creating experimentation dataset...")
train_copied = create_subset(
    os.path.join(data_dir, "train"),
    os.path.join(exp_dir, "train"),
    train_size
)

val_copied = create_subset(
    os.path.join(data_dir, "val"),
    os.path.join(exp_dir, "val"),
    val_size
)

test_copied = create_subset(
    os.path.join(data_dir, "test"),
    os.path.join(exp_dir, "test"),
    test_size
)

print(f"\nExperimentation dataset created:")
print(f"Train: {train_copied} images")
print(f"Validation: {val_copied} images")
print(f"Test: {test_copied} images")
print(f"Total: {train_copied + val_copied + test_copied} images")

'''

'\nimport os\nimport random\nimport shutil\nfrom pathlib import Path\nfrom tqdm import tqdm\n\n# Define paths\ndrive_root = \'/content/drive/MyDrive/Colab Notebooks/Deep Learning\'\ndata_dir = os.path.join(drive_root, "data")  # Where your splits are stored\nexp_dir = os.path.join(drive_root, "experimentation")  # Where to save smaller dataset\n\n# Define target sizes\ntrain_size = 1676\nval_size = 358\ntest_size = 358\n\n# Set random seed for reproducibility\nrandom.seed(42)\n\n# Create experimentation directories\nos.makedirs(os.path.join(exp_dir, "train"), exist_ok=True)\nos.makedirs(os.path.join(exp_dir, "val"), exist_ok=True)\nos.makedirs(os.path.join(exp_dir, "test"), exist_ok=True)\n\ndef create_subset(src_dir, dest_dir, target_size):\n    """Create a stratified subset of images maintaining class proportions."""\n\n    # Get all class folders\n    class_folders = [f for f in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, f))]\n\n    # Count total images\n    total_im

Creating experimentation dataset...

Creating subset in train: 100%|██████████| 202/202 [03:16<00:00,  1.03it/s]

Copied 1676 images to train

Creating subset in val: 100%|██████████| 202/202 [02:49<00:00,  1.19it/s]

Copied 358 images to val

Creating subset in test: 100%|██████████| 202/202 [02:48<00:00,  1.20it/s]

Copied 358 images to test

Experimentation dataset created:

- Train: 1676 images

- Validation: 358 images

- Test: 358 images

- Total: 2392 images


# 1. Setup and Imports

## 1.1 Importing libraries

In [24]:
!pip install keras-cv
!pip install keras-tuner

In [25]:
import keras
import keras_cv
import keras_tuner as kt
from keras_tuner import HyperModel, Hyperband
import tensorflow_hub as hub
import tensorflow as tf
from keras import Model, Sequential, layers
from keras.applications import ResNet50, MobileNetV2, EfficientNetB0, EfficientNetB7, DenseNet121, DenseNet169, DenseNet201
from keras.layers import (Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D,
                          Flatten, add, RandomFlip, RandomRotation, RandomBrightness,
                          RandomContrast, RandomTranslation, RandomZoom, LayerNormalization)

from keras.optimizers import SGD, Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, AUC, F1Score
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, EarlyStopping, Callback
from keras.preprocessing import image_dataset_from_directory
from pathlib import Path
from typing import Any, Dict, List, Tuple
import os
import matplotlib.pyplot as plt
import time
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import drive
from sklearn.model_selection import train_test_split
from shutil import copyfile
from tqdm import tqdm
import math

## 1.3 - Dataset Loading and Preprocessing

In [26]:
# Mount Google Drive
drive.mount('/content/drive')

# Set paths to your organized data directories

# Define base paths
drive_root = '/content/drive/MyDrive/Colab Notebooks/Deep Learning'

# Path to original metadata CSV (only used for EDA, not required for training)
metadata_path = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/species_metadata.csv'

# Configure for performance
AUTOTUNE = tf.data.AUTOTUNE

# Load metadata from CSV (optional, only for EDA visualizations)
df = pd.read_csv(metadata_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:

# Flag to control whether to use the full dataset or experimental subset
USE_SUBSET = True  # Set to False to use full datasets

# Choose the appropriate source directory based on the flag
if USE_SUBSET:
    data_source = os.path.join(drive_root, "experimentation")
    print("Using EXPERIMENTAL SUBSET for faster training")
else:
    data_source = os.path.join(drive_root, "data")
    print("Using FULL DATASET for final training")

# Load datasets directly from the chosen directory
train_ds = image_dataset_from_directory(
    os.path.join(data_source, "train"),
    label_mode="categorical",
    batch_size=64,
    image_size=(224, 224),
    shuffle=True,
    seed=42
)

val_ds = image_dataset_from_directory(
    os.path.join(data_source, "val"),
    label_mode="categorical",
    batch_size=64,
    image_size=(224, 224),
    shuffle=False
)

test_ds = image_dataset_from_directory(
    os.path.join(data_source, "test"),
    label_mode="categorical",
    batch_size=64,
    image_size=(224, 224),
    shuffle=False
)

# Store the class names BEFORE applying any transformations
num_classes = len(train_ds.class_names)
class_names = train_ds.class_names
print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names[:5]}... (showing first 5)")

# Configure datasets for performance
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

# Apply normalization preprocessing
normalization_layer = layers.Rescaling(1./255)

# Apply preprocessing to the datasets
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)

# Print dataset info
print("Dataset information:")
print(f"Train dataset: {train_ds}")
print(f"Validation dataset: {val_ds}")
print(f"Test dataset: {test_ds}")

# %% [markdown]
# ### Explanation of Dataset Loading
#
# We load our datasets directly from the organized directory structure:
#
# 1. **Directory Structure**:
#    - The `output_dir` points to a directory containing three subdirectories: "train", "val", and "test"
#    - Within each subdirectory, there's a folder for each family (class)
#    - Images for each family are stored in their respective family folders
#
# 2. **Metadata CSV** (Optional):
#    - The metadata CSV is loaded only for exploratory data analysis (EDA)
#    - It's not required for the actual model training since the class information comes from the directory structure
#    - If the CSV is unavailable, the code will still work but some visualizations might be limited
#
# 3. **Dataset Loading**:
#    - We use `image_dataset_from_directory` which automatically:
#      - Detects classes from folder names
#      - Loads images from each class folder
#      - Assigns the correct labels based on the folder structure
#    - Images are resized to 224×224 pixels to match the input size expected by the pre-trained models
#
# 4. **Dataset Optimization**:
#    - We apply normalization to scale pixel values from [0-255] to [0-1]
#    - We use prefetching to overlap data preprocessing and model execution for better performance

Using EXPERIMENTAL SUBSET for faster training
Found 1676 files belonging to 202 classes.
Found 358 files belonging to 202 classes.
Found 358 files belonging to 202 classes.
Number of classes: 202
Class names: ['accipitridae', 'acipenseridae', 'acroporidae', 'agamidae', 'agariciidae']... (showing first 5)
Dataset information:
Train dataset: <_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 202), dtype=tf.float32, name=None))>
Validation dataset: <_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 202), dtype=tf.float32, name=None))>
Test dataset: <_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 202), dtype=tf.float32, name=None))>


## 1.4 Check and verify GPU availability

In [28]:


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)




Fri Apr  4 11:22:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P0             28W /   70W |    8308MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [29]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


## 1.5 Very quick EDA

In [30]:

# Visualize family distribution
family_counts = df['family'].value_counts()
family_counts

,count
family,
dactyloidae,300
cercopithecidae,300
formicidae,291
carcharhinidae,270
salamandridae,270
...,...
cyprinodontidae,30
alligatoridae,30
balaenidae,30


# 2 - Helper Classes and Functions

In [31]:
# Define the Pipeline class (similar to Sequential but more explicitly for preprocessing)
#we use this because of pipeline augmentation class
class Pipeline(tf.keras.Model):
    def __init__(self, layers, name=None):
        super().__init__(name=name)
        self.layers_list = layers

    def call(self, inputs, training=False):
        x = inputs
        for layer in self.layers_list:
            x = layer(x, training=training)
        return x

def exp_decay_lr_scheduler(epoch, lr):
    """Exponential learning rate decay"""
    return float(lr * math.exp(-0.1 * epoch))


# %% [markdown]
# ### Explanation of Helper Classes and Functions
#
# 1. **Pipeline Class**:
#    - This is a custom implementation of a sequential pipeline for data augmentation
#    - Unlike the standard Keras Sequential, it explicitly passes the training flag to each layer
#    - This is important for layers that behave differently during training and inference
#
# 2. **exp_decay_lr_scheduler Function**:
#    - Implements exponential learning rate decay
#    - Reduces the learning rate by a factor of e^(-0.1) each epoch
#    - This helps the model converge more precisely as training progresses


# 3. Defining Data Augmentation Strategies

In [32]:
# We define different augmentation pipelines with varying levels of complexity.

# Define value range for augmentation
value_range = (0.0, 1.0)

# Define different augmentation pipelines

# 1. Simple Augmentation Pipeline
simple_augmentation = Pipeline(
    [
        RandomBrightness(factor=0.1, value_range=value_range),
        RandomFlip("horizontal"),
        RandomRotation(factor=0.1, fill_mode="reflect")
    ],
    name="simple_augmentation"
)

# 2. Medium Augmentation Pipeline
medium_augmentation = Pipeline(
    [
        RandomBrightness(factor=0.15, value_range=value_range),
        RandomFlip("horizontal"),
        RandomRotation(factor=0.15, fill_mode="reflect"),
        RandomContrast(factor=0.1, value_range=value_range),
        RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode="reflect")
    ],
    name="medium_augmentation"
)

# 3. Complex Augmentation Pipeline
complex_augmentation = Pipeline(
    [
        RandomBrightness(factor=0.2, value_range=value_range),
        RandomFlip("horizontal"),
        RandomFlip("vertical"),
        RandomRotation(factor=0.3, fill_mode="reflect"),
        RandomContrast(factor=0.2, value_range=value_range),
        RandomTranslation(height_factor=0.15, width_factor=0.15, fill_mode="reflect"),
        RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2), fill_mode="reflect"),

    ],
    name="complex_augmentation"
)
# Function to select augmentation based on complexity level
def get_augmentation(complexity="medium"):
    """
    Get augmentation pipeline based on desired complexity

    Args:
        complexity: One of "simple", "medium", or "complex"

    Returns:
        Selected augmentation pipeline
    """
    if complexity.lower() == "simple":
        return simple_augmentation
    elif complexity.lower() == "complex":
        return complex_augmentation
    else:
        # Default to medium
        return medium_augmentation

# %% [markdown]
# ### Explanation of Data Augmentation Strategies
#
# Data augmentation is crucial for improving model generalization, especially with limited data. We define three levels of augmentation complexity:
#
# 1. **Simple Augmentation**:
#    - Basic transformations: horizontal flips, small rotations, and brightness adjustments
#    - Conservative approach that preserves most image characteristics
#    - Best for: Models that are sensitive to distortion (like Vision Transformers)
#
# 2. **Medium Augmentation**:
#    - Adds contrast variation and image translation
#    - Moderate level of image distortion
#    - Best for: General purpose use with standard CNN architectures
#
# 3. **Complex Augmentation**:
#    - Adds vertical flips, larger rotations, zooming, and random erasing
#    - Aggressive augmentation for maximum diversity
#    - Best for: Smaller models that benefit from seeing more varied examples
#
# The visualizations show how these pipelines transform images:
# - The first visualization compares how each augmentation pipeline affects the same image
# - The second visualization shows multiple different variations created by the medium augmentation pipeline
#
# These transformations help our models learn robust features that generalize beyond the specific training examples, effectively increasing our training data diversity.



# 4. Architecture

## ResNet

In [33]:
class RareSpeciesResNet(Model):
    """
    Pre-trained ResNet-50 with data augmentation for rare species classification
    """

    def __init__(self, num_classes, unfrozen_layers=10, augmentation_type="medium"):
        """
        Initialization
        """
        super().__init__()

        self.num_classes = num_classes

        # Get the appropriate augmentation pipeline
        self.augmentation_layer = get_augmentation(augmentation_type)

        # Pre-trained architecture
        self.pre_trained_architecture = ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=(224, 224, 3)
        )

        # Freeze all layers except the last unfrozen_layers
        for layer in self.pre_trained_architecture.layers[:-unfrozen_layers]:
            layer.trainable = False

        # Final layers for classification
        self.global_pool_layer = GlobalAveragePooling2D()
        self.dense_layer = Dense(self.num_classes, activation="softmax")

    def call(self, inputs, training=False):
        """
        Forward call
        """
        # Apply augmentation only during training
        if training:
            x = self.augmentation_layer(inputs, training=training)
        else:
            x = inputs

        # Pre-trained model
        x = self.pre_trained_architecture(x)
        x = self.global_pool_layer(x)

        return self.dense_layer(x)


## MobileNet

In [34]:
class RareSpeciesMobileNet(Model):
    """
    Pre-trained MobileNetV2 with data augmentation for rare species classification
    """

    def __init__(self, num_classes, unfrozen_layers=15, augmentation_type="medium"):
        """
        Initialization
        """
        super().__init__()

        self.num_classes = num_classes

        # Get the appropriate augmentation pipeline
        self.augmentation_layer = get_augmentation(augmentation_type)

        # Pre-trained architecture
        self.pre_trained_architecture = MobileNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(224, 224, 3)
        )

        # Freeze all layers except the last unfrozen_layers
        for layer in self.pre_trained_architecture.layers[:-unfrozen_layers]:
            layer.trainable = False

        # Final layers for classification
        self.global_pool_layer = GlobalAveragePooling2D()
        self.dropout_layer = Dropout(0.2)
        self.dense_layer = Dense(self.num_classes, activation="softmax")

    def call(self, inputs, training=False):
        """
        Forward call
        """
        # Apply augmentation only during training
        if training:
            x = self.augmentation_layer(inputs, training=training)
        else:
            x = inputs

        # Pre-trained model
        x = self.pre_trained_architecture(x)
        x = self.global_pool_layer(x)
        x = self.dropout_layer(x, training=training)

        return self.dense_layer(x)

## Vit Model

In [35]:
class RareSpeciesViT(Model):
    """
    Pre-trained Vision Transformer with data augmentation for rare species classification
    """

    def __init__(self, num_classes, unfrozen_layers=2, augmentation_type="simple"):
        """
        Initialization
        """
        super().__init__()

        self.num_classes = num_classes

        # Get the appropriate augmentation pipeline
        self.augmentation_layer = get_augmentation(augmentation_type)

        # Load ViT from TensorFlow Hub
        self.pre_trained_architecture = hub.KerasLayer(
            "https://tfhub.dev/sayakpaul/vit_b16_fe/1",
            trainable=(unfrozen_layers > 0)
        )

        # Final layer for classification
        self.dense_layer = Dense(self.num_classes, activation="softmax")

    def call(self, inputs, training=False):
        """
        Forward call
        """
        # Apply augmentation only during training
        if training:
            x = self.augmentation_layer(inputs, training=training)
        else:
            x = inputs

        # Pre-trained model
        x = self.pre_trained_architecture(x)

        return self.dense_layer(x)

## EfficientNet

In [36]:
class RareSpeciesEfficientNet(Model):
    """
    Pre-trained EfficientNet with data augmentation for rare species classification
    """

    def __init__(self, num_classes, unfrozen_layers=10, augmentation_type="medium",
                 variant="B7"):
        """
        Initialization
        """
        super().__init__()

        self.num_classes = num_classes

        # Get the appropriate augmentation pipeline
        self.augmentation_layer = get_augmentation(augmentation_type)


        base_model_fn = EfficientNetB7


        # Pre-trained architecture
        self.pre_trained_architecture = base_model_fn(
            include_top=False,
            weights='imagenet',
            input_shape=(224, 224, 3)
        )

        # Freeze all layers except the last unfrozen_layers
        if unfrozen_layers > 0:
            for layer in self.pre_trained_architecture.layers[:-unfrozen_layers]:
                layer.trainable = False
        else:
            # Freeze all layers
            for layer in self.pre_trained_architecture.layers:
                layer.trainable = False

        # Final layers for classification
        self.global_pool_layer = GlobalAveragePooling2D()
        self.dropout_layer = Dropout(0.3)  # Higher dropout to combat overfitting
        self.dense_layer = Dense(self.num_classes, activation="softmax")

    def call(self, inputs, training=False):
        """
        Forward call
        """
        # Apply augmentation only during training
        if training:
            x = self.augmentation_layer(inputs, training=training)
        else:
            x = inputs

        # Pre-trained model
        x = self.pre_trained_architecture(x)
        x = self.global_pool_layer(x)
        x = self.dropout_layer(x, training=training)

        return self.dense_layer(x)

## Densenet

In [37]:
class RareSpeciesDenseNet(Model):
    """
    Pre-trained DenseNet with data augmentation for rare species classification
    """

    def __init__(self, num_classes, unfrozen_layers=10, augmentation_type="medium",
                 variant="201"):
        """
        Initialization

        Args:
            num_classes: Number of species classes to classify
            unfrozen_layers: Number of layers to unfreeze for fine-tuning
            augmentation_type: Type of augmentation pipeline to use
            variant: DenseNet variant (121, 169, or 201)
        """
        super().__init__()

        self.num_classes = num_classes

        # Get the appropriate augmentation pipeline
        self.augmentation_layer = get_augmentation(augmentation_type)


        base_model_fn = DenseNet201


        # Pre-trained architecture
        self.pre_trained_architecture = base_model_fn(
            include_top=False,
            weights='imagenet',
            input_shape=(224, 224, 3)
        )

        # Freeze all layers except the last unfrozen_layers
        if unfrozen_layers > 0:
            total_layers = len(self.pre_trained_architecture.layers)
            for layer in self.pre_trained_architecture.layers[:-unfrozen_layers]:
                layer.trainable = False
            for layer in self.pre_trained_architecture.layers[-unfrozen_layers:]:
                layer.trainable = True
            print(f"Unfrozen {unfrozen_layers} of {total_layers} layers in DenseNet{variant}")
        else:
            # Freeze all layers
            for layer in self.pre_trained_architecture.layers:
                layer.trainable = False
            print(f"All layers in DenseNet{variant} are frozen")

        # Final layers for classification
        self.global_pool_layer = GlobalAveragePooling2D()
        self.dropout_layer = Dropout(0.3)  # Higher dropout for regularization
        self.dense_layer = Dense(self.num_classes, activation="softmax")

    def call(self, inputs, training=False):
        """
        Forward call

        Args:
            inputs: Input tensor of shape [batch_size, height, width, channels]
            training: Boolean indicating whether in training mode

        Returns:
            Class probability predictions
        """
        # Apply augmentation only during training
        if training:
            x = self.augmentation_layer(inputs, training=training)
        else:
            x = inputs

        # Pre-trained model
        x = self.pre_trained_architecture(x)
        x = self.global_pool_layer(x)
        x = self.dropout_layer(x, training=training)

        return self.dense_layer(x)

# 5. Model Training Function

In [38]:

class RareSpeciesHyperModel(HyperModel):
    """
    HyperModel for rare species classification that works with existing model classes
    """
    def __init__(self, num_classes, model_type='resnet'):
        self.num_classes = num_classes
        self.model_type = model_type

    def build(self, hp):
        """
        Build model with hyperparameters
        """
        # Tune the augmentation type
        augmentation_type = hp.Choice(
            'augmentation_type',
            values=['simple', 'medium', 'complex'],
            default='medium'
        )

        # Tune the number of unfrozen layers
        unfrozen_layers = hp.Int(
            'unfrozen_layers',
            min_value=0,
            max_value=20,
            default=5,
            step=1
        )

        # Create model based on type
        if self.model_type == 'resnet':
            model = RareSpeciesResNet(
                num_classes=self.num_classes,
                unfrozen_layers=unfrozen_layers,
                augmentation_type=augmentation_type
            )
        elif self.model_type == 'mobilenet':
            model = RareSpeciesMobileNet(
                num_classes=self.num_classes,
                unfrozen_layers=unfrozen_layers,
                augmentation_type=augmentation_type
            )
        elif self.model_type == 'vit':
            model = RareSpeciesViT(
                num_classes=self.num_classes,
                unfrozen_layers=unfrozen_layers > 0,  # ViT uses boolean for trainable
                augmentation_type=augmentation_type
            )
        elif self.model_type == 'efficientnet':
            # Additional parameter for EfficientNet variant
            variant = hp.Choice(
                'efficientnet_variant',
                values=['B0', 'B1', 'B2', 'B3'],
                default='B0'
            )

            model = RareSpeciesEfficientNet(
                num_classes=self.num_classes,
                unfrozen_layers=unfrozen_layers,
                augmentation_type=augmentation_type,
                variant=variant
            )
        elif self.model_type == 'densenet':
            # Additional parameter for DenseNet variant
            variant = hp.Choice(
                'densenet_variant',
                values=['121', '169', '201'],
                default='121'
            )

            model = RareSpeciesDenseNet(
                num_classes=self.num_classes,
                unfrozen_layers=unfrozen_layers,
                augmentation_type=augmentation_type,
                variant=variant
            )
        else:
            raise ValueError(f"Unsupported model type: {self.model_type}")

        # Tune batch size
        batch_size = hp.Choice(
            'batch_size',
            values=[8, 16, 32, 64],
            default=32
        )

        # Tune learning rate
        learning_rate = hp.Float(
            'learning_rate',
            min_value=1e-5,
            max_value=1e-2,
            sampling='log',
            default=1e-3
        )

        # Tune optimizer
        optimizer_type = hp.Choice(
            'optimizer',
            values=['sgd', 'adam'],
            default='sgd'
        )

        if optimizer_type == 'sgd':
            # Tune momentum for SGD
            momentum = hp.Float(
                'momentum',
                min_value=0.7,
                max_value=0.99,
                default=0.9,
                step=0.05
            )

            # Tune weight decay for SGD
            weight_decay = hp.Float(
                'weight_decay',
                min_value=1e-6,
                max_value=1e-3,
                sampling='log',
                default=1e-4
            )

            optimizer = SGD(
                learning_rate=learning_rate,
                momentum=momentum,
                weight_decay=weight_decay,
                nesterov=True
            )
        else:
            optimizer = Adam(learning_rate=learning_rate)

        # Compile model
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=[
                'accuracy',
                tf.keras.metrics.AUC(),
                tf.keras.metrics.F1Score(average='macro', name='f1_score')
            ]
        )

        # Store batch size as a model attribute
        model.batch_size = batch_size

        return model

def run_hyperband_tuning(train_ds, val_ds, test_ds, model_type='densenet', max_epochs=30,
                         num_classes=400, project_name='rare_species'):
    """
    Run Hyperband hyperparameter tuning for a specific model type
    """
    # Create directory for results
    results_dir = f'hyperband_results/{model_type}'
    os.makedirs(results_dir, exist_ok=True)

    # Initialize the HyperModel
    hypermodel = RareSpeciesHyperModel(
        num_classes=num_classes,
        model_type=model_type
    )

    # Configure the Hyperband tuner
    tuner = Hyperband(
        hypermodel,
        objective='val_accuracy',
        max_epochs=max_epochs,
        factor=3,
        directory=results_dir,
        project_name=f'{project_name}_{model_type}'
    )

    # Print search space summary
    tuner.search_space_summary()

    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    # CSV Logger to track training progress
    csv_logger = tf.keras.callbacks.CSVLogger(
        f'{results_dir}/{model_type}_tuning.csv',
        append=True
    )

    # Run the search
    print(f"\nStarting Hyperband tuning for {model_type} model...")
    print(f"Results will be saved to {results_dir}")

    start_time = time.time()

    # Standard tuner search with only standard callbacks
    tuner.search(
        train_ds,
        validation_data=val_ds,
        callbacks=[early_stopping, csv_logger]
    )

    tuning_time = time.time() - start_time
    print(f"Hyperband tuning completed in {tuning_time:.2f} seconds")

    # Get best hyperparameters
    best_hp = tuner.get_best_hyperparameters(1)[0]

    print(f"\nBest hyperparameters:")
    for param, value in best_hp.values.items():
        print(f"- {param}: {value}")

    # Build model with best hyperparameters
    best_model = tuner.hypermodel.build(best_hp)

    # Save best hyperparameters to JSON
    import json
    with open(f'{results_dir}/best_hyperparameters.json', 'w') as f:
        json.dump(best_hp.values, f, indent=2)

    # Train the model for a few epochs to evaluate metrics
    print("\nTraining model with best hyperparameters to collect metrics...")

    # Getting the batch size
    batch_size = best_hp.values.get('batch_size', 32)

    # Simple metrics collection - we'll use a regular callback
    class SimpleMetricsCollector(tf.keras.callbacks.Callback):
        def __init__(self):
            super().__init__()
            self.metrics = {}
            self.start_time = time.time()

        def on_train_end(self, logs=None):
            self.metrics['training_time'] = time.time() - self.start_time

    metrics_collector = SimpleMetricsCollector()

    # Train for a few epochs to get stable metrics
    history = best_model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=5,  # Just a few epochs to get stable metrics
        batch_size=batch_size,
        callbacks=[metrics_collector]
    )

    # Evaluate on train and validation sets
    train_metrics = best_model.evaluate(train_ds, return_dict=True, verbose=0)
    val_metrics = best_model.evaluate(val_ds, return_dict=True, verbose=0)

    # Calculate overfitting score
    overfitting = train_metrics['accuracy'] - val_metrics['accuracy']

    # Print metrics table
    print("\nMetrics:")
    print("CV F1 Macro Val | CV F1 Macro Train | Accuracy Val | Accuracy Train | Loss Val | Loss Train | Overfitting | Time (s)")
    print(f"{val_metrics['f1_score']:.4f} | {train_metrics['f1_score']:.4f} | " +
          f"{val_metrics['accuracy']:.4f} | {train_metrics['accuracy']:.4f} | " +
          f"{val_metrics['loss']:.4f} | {train_metrics['loss']:.4f} | " +
          f"{overfitting:.4f} | {metrics_collector.metrics['training_time']:.2f}")

    # Create a DataFrame with metrics
    metrics_dict = {
        'val_f1_score': val_metrics['f1_score'],
        'train_f1_score': train_metrics['f1_score'],
        'val_accuracy': val_metrics['accuracy'],
        'train_accuracy': train_metrics['accuracy'],
        'val_loss': val_metrics['loss'],
        'train_loss': train_metrics['loss'],
        'overfitting': overfitting,
        'training_time': metrics_collector.metrics['training_time']
    }

    # Add hyperparameters to the metrics
    for param, value in best_hp.values.items():
        metrics_dict[param] = value

    metrics_df = pd.DataFrame([metrics_dict])

    # If test_ds is provided, evaluate on it
    if test_ds is not None:
        print("\nEvaluating on test set...")
        test_metrics = best_model.evaluate(test_ds, return_dict=True, verbose=1)
        metrics_dict['test_accuracy'] = test_metrics['accuracy']
        metrics_dict['test_f1_score'] = test_metrics['f1_score']
        metrics_dict['test_loss'] = test_metrics['loss']

        print(f"\nTest metrics:")
        print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
        print(f"- F1 Score: {test_metrics['f1_score']:.4f}")
        print(f"- Loss: {test_metrics['loss']:.4f}")

    # Save metrics to CSV
    metrics_df.to_csv(f'{results_dir}/best_model_metrics.csv', index=False)

    return best_hp, best_model, tuner, metrics_df

def compare_all_models(train_ds, val_ds, test_ds, num_classes=400, max_epochs=30):
    """
    Run Hyperband tuning for all models and compare results
    """
    model_types = ['resnet', 'mobilenet', 'vit', 'efficientnet', 'densenet']
    all_results = {}

    # Run tuning for each model type
    for model_type in model_types:
        print(f"\n{'-'*80}")
        print(f"TUNING {model_type.upper()}")
        print(f"{'-'*80}")

        best_hp, best_model, tuner, trials_df = run_hyperband_tuning(
            train_ds=train_ds,
            val_ds=val_ds,
            model_type=model_type,
            max_epochs=max_epochs,
            num_classes=num_classes
        )

        # Evaluate on test set
        test_results = best_model.evaluate(test_ds, return_dict=True)

        # Store results
        all_results[model_type] = {
            'hyperparameters': best_hp.values,
            'model': best_model,
            'tuner': tuner,
            'trials_df': trials_df,
            'test_metrics': test_results
        }

    # Create comparison table
    comparison_data = []
    for model_type, result in all_results.items():
        # Get the best trial
        best_trial = result['trials_df'].sort_values('overfitting').iloc[0]

        comparison_data.append({
            'Model': model_type,
            'Val F1': best_trial['val_f1'],
            'Train F1': best_trial['train_f1'],
            'Val Acc': best_trial['val_accuracy'],
            'Train Acc': best_trial['train_accuracy'],
            'Val Loss': best_trial['val_loss'],
            'Train Loss': best_trial['train_loss'],
            'Overfitting': best_trial['overfitting'],
            'Time (s)': best_trial['time'],
            'Test Acc': result['test_metrics']['accuracy'],
            'Test F1': result['test_metrics']['f1_score']
        })

    # Convert to DataFrame
    comparison_df = pd.DataFrame(comparison_data)

    # Sort by overfitting (lower is better)
    comparison_df = comparison_df.sort_values('Overfitting')

    print("\nModel Comparison (sorted by least overfitting):")
    print(comparison_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

    # Save comparison to CSV
    comparison_df.to_csv('model_comparison.csv', index=False)

    # Find the model with least overfitting
    best_model_type = comparison_df.iloc[0]['Model']
    print(f"\nModel with least overfitting: {best_model_type.upper()}")

    return all_results, comparison_df

# 6. Model Training and Evaluation

In [ ]:
# Run the ultra-simple hyperband tuning
best_hp, best_model, tuner, metrics_df = run_hyperband_tuning(
    train_ds=train_ds,
    val_ds=val_ds,
    test_ds=test_ds,  # Optional, pass None if not ready to evaluate on test set
    model_type='densenet',  # Change to any model type
    max_epochs=30,
    num_classes=202
)

# After tuning, train the best model for more epochs if desired
batch_size = best_hp.values.get('batch_size', 32)
print(f"\nTraining the best model with batch size {batch_size} for 60 epochs...")

# Train for more epochs
history = best_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=60,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            f'best_{model_type}_model.keras',
            save_best_only=True,
            monitor='val_loss'
        )
    ]
)

# Plot training history
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.savefig(f'{model_type}_training_history.png')
plt.show()

Trial 3 Complete [00h 02m 10s]
val_accuracy: 0.05865921825170517

Best val_accuracy So Far: 0.20111732184886932
Total elapsed time: 00h 18m 55s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
simple            |complex           |augmentation_type
8                 |7                 |unfrozen_layers
169               |169               |densenet_variant
8                 |8                 |batch_size
0.00019349        |0.0011555         |learning_rate
sgd               |adam              |optimizer
0.95              |0.85              |momentum
0.00012173        |0.00012223        |weight_decay
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Unfrozen 8 of 707 layers in DenseNet169
Epoch 1/2
27/27 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.0095 - auc: 0.5070 - f1_score: 0.0040 - loss: 5.7